# Про методы импутации

Проблема такая: в исходных данных часто бывают NaNы, то есть часть данных может просто отсутствовать. Это сильно влияет на качество обучения сети. С этим можно бороться разными способами. В первую очередь, образцы, для которых часть данных отутствует, можно просто выкинуть, но тогда теряется значимая информация.

Другие методы:
1) игнорировать NaNы. Пока не понимаю, почему этот способ не универсальный и зачем нужно что-то придумывать и импутировать.
2) некоторые алгоритмы умеют подбирать, что вставлять, based on the training loss reduction.

Судя по всему, некоторые алгоритмы просто не умеют воспринимать NaNы, и приходится вставлять их до этапа training-a.<br/>
3) вставлять среднее. Это просто, но не учитывает никаких закономерностей распределения значений. Плюс не работает с категориальными значениями (пол, наличие/отсутствие чего-то). Часто, кажется, может вносить bias.<br/>
4) вставлять самое частое значение или константу. Это работает с категориальными значениями. Но это не учитывает закономерностей и вносит bias.<br/>
5) k-NN (кун не нужны). Берет k ближайших соседей (подсчет feature similarity), от них берет weighted average. Это затратно, потому что в память приходится закачивать весь dataset. Еще пишут, что он чувствительный к outlierам. Пока не понимаю почему. Видимо, это играет роль не на стадии выбора соседей, а на стадии подсчета average. И тут мы знаем, как с ними бороться.<br/>
6) joint modeling (JM) 
*#когда придумал подход, гуляя с собакой в парке, а потом наткнулся на статью о нем (где моя Нобелевка).*
Это короче мы строим многомерное распределение по дате, а потом Марковские цепи, рас рас, и мы вставляем значения так, чтобы они под наше многомерное распределения. Это нормально работает, если дату можно нормально апроксимировать знакомым распределением. <br/>
7) MICE (Imputation Using Multivariate Imputation by Chained Equation)
Не очень пока понимаю, как оно работает, но суть в том, что он итеративный, то есть несколько раз подбирает каждую вставку. Судя по всему, он сначала вставляет значение, смотрит, какое там локальное распределение получается, в зависимости от него снова подбирает параметр. А может, я что-то не так поняла. Оно хорошо работает, если нет возможности построить адекватное многомерное распределение. <br/>
8) Машин лернингом их подбирать. 


В нашем случае способ 1) не подходит, потому что авторы позиционируют scScope как штуку, позволяющую работать со слабо экспрессирующими генами. То есть NaNов там много, и если их игнорировать, будет не очень. По крайней мере от этого пропадет вся суть программы. Ее основная идея во множественной коррекции импутации. Пункт 2) - по той же причине не наш случай. 
Пробовать надо 5-7, наверное. Хотя вставлять итеративный MICE в итеративную архитектуру - это матрешка. 

# Batch effect correction

Sources of variability in experimentally derived data include measurement error in addition to the physical phenomena of interest. This measurement error is a combination of systematic components, originating from the measuring instrument and random measurement errors. Several novel biological technologies, such as mass cytometry and single-cell RNA-seq (scRNA-seq), are plagued with systematic errors that may severely affect statistical analysis if the data are not properly calibrated.


Итак, методы борьбы с этим в случае с scRNA-seqом:


1) SVA метод (Surrogate variable analysis)
https://bioconductor.org/packages/devel/bioc/vignettes/sva/inst/doc/sva.pdf
We model the data as a combination of known variables of interest, known adjustment variables (known batches) and unknown and unmeasured artifacts. A simple version of this model might relate gene expression for gene ion sample j(gij) to the phenotype for that sample yj, the known batch variable for that sample (aj)and an unknown artifact on that sample 
![](http://www.tran-med.com/article/2016/2411-2917-2-1-3/img_1.png)
, where gij is the gene expression, bi0stands for the baseline expression, bi1yj is the phenotype effect for sample j (like age, gender…), ciaj stands for the known batch factor, diuj represent the unknown artifact, and the error term eij is the measurement error.

2) Баесовский COMBAT метод. 
![](https://imgur.com/eYkrLXS.png)

3) PCA метод
Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. The number of principal components is less than or equal to the number of original variables. This transformation is defined such that the first principal component has the largest possible variance (that is, accounts for as much of the variability in the data as possible), and each succeeding component in turn has the highest variance possible under the constraint that it is orthogonal to (i.e., uncorrelated with) the preceding components. The principal components are the eigenvectors of the covariance matrix (which is also symmetric), so they are orthogonal.

PCA method could be intuitively used in correcting for batch effects in microarray or RNA-seq data. First we use the principal component method to infer the data variance in different axes. As the greatest source of variance in gene expression studies is nearly always across batches rather than across biological groups [5], it would be straightforward to view the batch effect mostly in the first 2 PC axes. We have used PCA method to remove batch effect in RNA-seq data, and have showed the first 2 PC axes for each method before and after correcting for the batch effect, obtaining a direct impression of the performance of each method.

4) Что я хочу попробовать - остаточные нейронные сети
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5870543/

5) Что делают они - используют обучаемую матрицу batch-коррекции
![](https://imgur.com/AOVNDgR.png)
Где B - та самая матрица. u - вектор-индикатор батч эффекта (One hot encoding, ненулевое значение обозначает бэтч xc). r - ReLU юнит.

Информация о принадлежности к тому или иному батчу хранится в exp_batch_idx. 


In [0]:
exp_batch_id_shape = exp_batch_idx.get_shape().as_list()
exp_batch_dim = exp_batch_id_shape[1]
with tf.variable_scope('batch_effect_removal'):
  batch_effect_para_weight = _variable_with_weight_decay('batch_effect_weight',
                                                        [exp_batch_dim,
                                                        input_dim],
                                                        stddev=0, wd=0)

  batch_effect_removal_layer = tf.matmul(
      exp_batch_idx, batch_effect_para_weight)

#Аннотация кода

In [0]:
# scScope is a deep-learning based approach designed to identify cell-type composition from large-scale scRNA-seq profiles.

"""Добрый день, друзья. Сегодня мы познакомимся с потрясающим миром tensorflow (TF). TF построен на двух важнейших
элементах: Tensor'ы и Flow. Подружимся с ними поближе:

Tensor - n-размерная матрица, содержащая в себе элементы. В общем, тензор - математический объект, аналогичный векторам,
но более обобщенный, представленный эрреем компонетнов, являющихся функциями координат пространства. Ранг тензора - количес
тво измерений тензора (= размерность ведь?). Тензоры - главные элементы TF, они являются узлами вычислительного графа. Таким образом, граф
представляет собой тензоры, линейно соединенные друг с другом взаимоотношением математических функций.

Поток данных по тензорам - Flow.

За подробностями - https://towardsdatascience.com/a-beginner-introduction-to-tensorflow-part-1-6d139e038278

У TensorFlow существует 2 версии: TF CPU и TF GPU. Последний работает не на всех компьютерах, а только на ограниченном
наборе видеокарт, поддерживающих технологию вычисления Nvidia CUDA. Нейронки на GPU работают и обучаются
в десятки раз быстрее, чем на CPU. Поэтому всегда предпочтительней работать с ГПУшной версией.

Разберем импорты:"""


# __future__ содержит в себе служебные функции. Например, absolute_import меняет поведение import так, чтобы
# правильно импортировались подмодули по их абсолютному пути (в данном случае .ops)
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# Tensorflow CPU и TF GPU импортируются идентично, в зависимости от того, какая версия установлена в энвайроменте
import tensorflow as tf
import numpy as np
import random
import time
# ops содержит вспомогательные функции, которые используются для освобождения памяти CPU и GPU, а также параллелизации
# процессов
from .ops import average_gradients, _variable_with_weight_decay, _variable_on_cpu
# PhenoGraph is a clustering method designed for high-dimensional single-cell data. It works by creating a graph
# ("network") representing phenotypic similarities between cells and then identifying communities in this graph.
import phenograph
# Метод кластеризации по K-средним с помощью одного класса
from sklearn.cluster import KMeans


In [0]:
def train_large(train_data_path,
                file_num,
                cell_size,
                gene_size,
                latent_code_dim,
                exp_batch_idx=0,
                use_mask=True,
                batch_size=64,
                max_epoch=100,
                epoch_per_check=100,
                T=2,
                encoder_layers=[],
                decoder_layers=[],
                learning_rate=0.0001,
                beta1=0.05,
                num_gpus=1
                ):
    '''
    scScope training:
	  This function is used to train the scScope model on gene expression data
    Parameters:
      train_data_path:      File path of multiple small gene expression files. Each file is a cell_size * gene_size matrix stored in *.npy format.
                                                    Files are named in "batch_0.npy", "batch_1.npy", ...
      file_num:             Number of gene expression files in "train_data_path".
      cell_size:            Cell numbers in each expression file. All files should include the same number of cells.
      gene_size:            Gene numbers in each expression file. All files should include the same number of genes.
      exp_batch_idx:        Number of experimental batches in the sequencing. if exp_batch_idx = 0, no batch information need to provide.
                                                    Otherwise, experimental batch labels are stored in "exp_batch_label_0.npy", "exp_batch_label_1.npy", ..., corresponding to each data batch file.
                                                    In each file, experimental batch labels are stored in an n * batch_num matrix in one-hot format. Experimental batch labels and data batch files
                                                    are in the same directory.
      latent_code_dim:      The feature dimension outputted by scScope.
      batch_size:           Number of cells used in each training iteration.
      max_epoch:            Maximal epoch used in training.
      epoch_per_check:      Step to display current loss.
      T:                    Depth of recurrence used in deep learning framework.
      use_mask:             Flag indicating whether to use only non-zero entries in calculating losses.
      learning_rate:        Step length in gradient descending algorithm.
      beta1:                The beta1 parameter in AdamOptimizer.
      num_gpus:             Number of gpus used for training in parallel.
    Output:
      model: a dataframe of scScope outputs with keys:
                    'latent_code_session':      tensorflow session used in training.
                    'test_input':               tensorflow dataholder for test data.
                    'test_exp_batch_idx':       tensorflow dataholder for experimental batch label.
                    'imputated_output':         imputed gene expressions.
                    'latent_code':              latent features by scScope.
                    'removed_batch_effect':     correcting layer learning by scScope.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''
    # Мы будем тренить модель на одной gpu, поэтому считайте num_gpus = 1
    # Здесь задается размер батча - пачки данных, которую мы будем единовременно подавать модели для обучения. 
    # Если сразу подавать модели все данные (batch_size = dataset_size), она будет занимать чрезмерно много памяти, 
    # поэтому делим на батчи. Есть так называемый stochaistic gradient descent, где мы подаем на вход лишь 1 
    # instance, это другой экстрим. В таком случае модель будет обучаться дольше и хуже. Нужно подобрать такой размер
    # батча, чтобы не занимать слишком много оперативки, и нформации в 1 батче хватало, чтобы нейронка смогла выявить 
    # закономерности.
    batch_size = int(batch_size * num_gpus)
    # Learning rate - гиперпараметр, который определяет, как сильно мы будем изменять веса и баясы по отношению к loss. 
    # Чем больше learning rate, тем быстрее loss (отклонение от правильных лейблов) стремится к 0, однако быстро не всегда хорошо.
    # Нейронка может овершутнуть и пропустить локальный минимум, и в итоге к нему не сойтись, просто потому что шаг будет слишком большой. 
    # Чтобы побороть это, мы используем не слишком большой learning rate, а также ADAM optimiser, в котором по мере обучения корректируется 
    # штраф, накладываемый на ноды таким образом, чтобы не перепрыгнуть минимум.
    """Почему нельзя сделать как при титровании? я читала, что вместо лосс используется квадрат лосса, чтобы не минимизировать 
    функцию, которая может быть и отрицательной, но с этим можно и по-другому бороться. А дальше задать большой learning rate, 
    задетектить момент, когда знак ошибки меняется, вернуться к предыдущим weights, уменьшить learning rate и т.д."""
    learning_rate = learning_rate * num_gpus

    if exp_batch_idx == 0:
        exp_batch_idx_input = np.zeros((cell_size, 1))
        consider_exp_batch = False
    else:
        consider_exp_batch = True

    with tf.Graph().as_default(), tf.device('/cpu:0'):
    # Тензоры в тензорфлоу реализуются плейсхолдерами. Placeholder - объект, которому присваевается значение или 
    # тензор по мере прохождения информации по вычислительному графу. Здесь мы задаем плейсхолдер для трейна, который 
    # имеет размер (batch_size, gene_size(количество генов)), а также задаем плейсхолдер для реальных лейблов
        train_data = tf.placeholder(
            tf.float32, [batch_size, gene_size])
        exp_batch_idx = tf.placeholder(tf.float32,
                                       [batch_size, exp_batch_idx])

        # Create an optimizer that performs gradient descent.
        opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1)
      
        # Calculate the gradients on models deployed on each GPU then summarized the gradients.
        # Градиенты - векторы, элементами которых являются частные производные функции предсказания по каждому 
        # направлению (???количества генов??? это как). Удобно представлять, что наша модель пытается выучить функцию размерности 
        # gene_size. Градиент просто указывает, в какую сторону нужно менять веса, чтобы аппроксимация стала лучше
        tower_grads = []
        tower_grads2 = []

        with tf.variable_scope(tf.get_variable_scope()):
        # Здесь строится вычислительный граф, состоящий из всех слоев и операций
            for i in range(num_gpus):

                print('Building Computational Graph on GPU-' + str(i))

                with tf.device('/gpu:%d' % (i + 1)):

                    with tf.name_scope('%s_%d' % ('tower', i)) as scope:

                        itv = int(batch_size / num_gpus)

                        if i == 0:

                            re_use_flag = False

                        else:

                            re_use_flag = True

                        loss = tower_loss(scope,
                                          train_data[(i) *
                                                     itv:(i + 1) * itv, :],
                                          use_mask,
                                          latent_code_dim,
                                          T,
                                          encoder_layers,
                                          decoder_layers,
                                          exp_batch_idx[(
                                              i) * itv:(i + 1) * itv, :],
                                          re_use_flag)

                        tf.get_variable_scope().reuse_variables()

                        t_vars = tf.trainable_variables()

                        inference_para = [
                            var for var in t_vars if 'inference' in var.name]
                        grads = opt.compute_gradients(loss, inference_para)
                        tower_grads.append(grads)

                        if consider_exp_batch:
                            exp_batch_effect_para = [
                                var for var in t_vars if 'batch_effect_removal' in var.name]
                            grads2 = opt.compute_gradients(
                                loss, exp_batch_effect_para)
                            tower_grads2.append(grads2)

        # Summarize gradients from multiple GPUs.
        grads = average_gradients(tower_grads)
        apply_gradient_op = opt.apply_gradients(grads)
        train_op = apply_gradient_op

        if consider_exp_batch:
            grads2 = average_gradients(tower_grads2)
            apply_gradient_op2 = opt.apply_gradients(grads2)
            train_op2 = apply_gradient_op2

        init = tf.global_variables_initializer()

        # Configuration of GPU.
        # Вычислительный граф деплоится на GPU
        config_ = tf.ConfigProto()

        config_.gpu_options.allow_growth = True

        config_.allow_soft_placement = True
        # По заданному вычислительному графу запускается сессия 
        # Короткий урок, чтобы понять, что такое сессия: https://danijar.com/what-is-a-tensorflow-session/
        sess = tf.Session(config=config_)

        sess.run(init)

        reconstruction_error = []
        # Начинается счетчик времени обучения
        start = time.time()
        
        # Начинается итерация. max_epoch - количество полных итераций по всей дате
        for step in range(1, max_epoch + 1):

            for file_count in range(file_num):
                # Чтобы не засорять оперативку, нужные файлы подгружаются только внутри цикла, 
                # и вне его сразу дампятся
                train_data_real_val = np.load(
                    train_data_path + '/batch_' + str(file_count) + '.npy')
                if exp_batch_idx > 0:
                    exp_batch_idx_input = np.load(
                        train_data_path + '/exp_batch_label_' + str(file_count) + '.npy')

                total_data_size = np.shape(train_data_real_val)[0]
                total_sample_list = list(range(total_data_size))

                total_cnt = total_data_size / (batch_size)

                for itr_cnt in range(int(total_cnt)):

                    sel_pos = random.sample(total_sample_list, batch_size)

                    cur_data = train_data_real_val[sel_pos, :]
                    cur_exp_batch_idx = exp_batch_idx_input[sel_pos, :]
                # Важнейший метод для Session - run. Он запускает нужный фрагмент вычислительного графа 
                # для вычислить каждый тензор в train_op. Данные, по которым запускается вычислительный граф
                # содержатся в словаре feed_dict. В данном случае, это наша train data и реальные лейблы (для вычисления лосса)
                    sess.run(train_op,
                             feed_dict={train_data: cur_data,
                                        exp_batch_idx: cur_exp_batch_idx})
                # Здесь исправляется бэтч эффект
                    if consider_exp_batch:
                        sess.run(train_op2,
                                 feed_dict={train_data: cur_data,
                                            exp_batch_idx: cur_exp_batch_idx})

                if step % epoch_per_check == 0 and step > 0:

                    all_input = tf.placeholder(
                        tf.float32, [np.shape(train_data_real_val)[0], np.shape(train_data_real_val)[1]])
                    exp_batch_idx_all = tf.placeholder(
                        tf.float32, [np.shape(exp_batch_idx_input)[0], np.shape(exp_batch_idx_input)[1]])

                    layer_output, train_latent_code, _ = Inference(
                        all_input, latent_code_dim, T, encoder_layers, decoder_layers, exp_batch_idx_all, re_use=True)

                    train_code_val, layer_output_val = sess.run(
                        [train_latent_code[-1], layer_output[-1]],
                        feed_dict={all_input: train_data_real_val, exp_batch_idx_all: exp_batch_idx_input})

                    mask = np.sign(train_data_real_val)
                    recon_error = np.linalg.norm(np.multiply(mask, layer_output_val) - np.multiply(
                        mask, train_data_real_val)) / np.linalg.norm(np.multiply(mask, train_data_real_val))
                    reconstruction_error.append(recon_error)
                    print("Finisheded epoch：" + str(step))
                    print('Current reconstruction error is: ' + str(recon_error))
                    # Здесь на каком-то задаваемом интервале выдается информация об обучении, а именно номер законченной эпохи, а также
                    # текущая ошибка реконструкции (отклонение от того, что подавалось на вход)

                    if len(reconstruction_error) >= 2:
                        if (abs(reconstruction_error[-1] - reconstruction_error[-2]) / reconstruction_error[-2] < 1e-3):
                            break

        model = {}
        # После окончания обучения заранее строится модель для теста (она подается на выход этой функции)
        test_data_holder = tf.placeholder(
            tf.float32, [None, gene_size])
        test_exp_batch_idx = tf.placeholder(
            tf.float32, [None, exp_batch_idx])

        test_layer_out, test_latent_code, removed_batch_effect = Inference(
            test_data_holder, latent_code_dim, T, encoder_layers, decoder_layers, test_exp_batch_idx, re_use=True)

        model['latent_code_session'] = sess
        model['test_input'] = test_data_holder
        model['test_exp_batch_idx'] = test_exp_batch_idx
        model['imputated_output'] = test_layer_out
        model['latent_code'] = test_latent_code
        model['removed_batch_effect'] = removed_batch_effect

        duration = time.time() - start
        print('Finish training ' + str(len(train_data)) + ' samples after ' + str(
            step) + ' epochs. The total training time is ' +
            str(duration) + ' seconds.')

        return model

"""Это уже после обучения запускается тест"""
def predict_large(train_data_path,
                  file_num,
                  model,
                  exp_batch_idx=0):
    '''
    Output the latent feature and imputed sequence for large scale dataset after training the model.
    Parameters:
            train_data_path:    The same data path as in "train_large()".
            file_num:           Number of data files in train_data_path.
            exp_batch_idx:      Number of experimental batches in sequencing. If exp_batch_idx=0, the function is run without batch correction.
            model:              The pre-trained model by "train_large()".
    Output:
            Latent features and imputed genes for each data file.
            For data file "batch_i.npy",  corresponding latent features and imputed gene expressions are stored in
            "feature_i.npy" and "imputation_i.npy" files respectively in the same directory.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''
    for file_count in range(file_num):

        train_data = np.load(
            train_data_path + '/batch_' + str(file_count) + '.npy')
        if exp_batch_idx > 0:
            batch_effect = np.load(
                train_data_path + '/exp_batch_label_' + str(file_count) + '.npy')
        else:
            batch_effect = []
        latent_fea, output_val, predicted_batch_effect = predict(
            train_data, model, batch_effect=batch_effect)
        np.save(train_data_path + '/feature_' +
                str(file_count) + '.npy', latent_fea)
        np.save(train_data_path + '/imputation_' +
                str(file_count) + '.npy', output_val)


def predict(test_data, model, batch_effect=[]):
    '''
    Make predications using the learned scScope model.
	
    Parameter:
            test_data:      gene expression matrix need to make prediction.
            model:          pre-trained scScope model.
    Output:
            latent_fea:             scScope features for inputted gene expressions.
            output_val:             gene expressions with imputations.
            predicted_batch_effect: batch effects inferenced by scScope, if experimental batches exist.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''

    sess = model['latent_code_session']
    test_data_holder = model['test_input']
    test_exp_batch_idx_holder = model['test_exp_batch_idx']
    output = model['imputated_output']
    latent_code = model['latent_code']
    removed_batch_effect = model['removed_batch_effect']
    if len(batch_effect) == 0:
        batch_effect_idx = np.zeros((np.shape(test_data)[0], 1))
    else:
        batch_effect_idx = batch_effect

    for i in range(len(latent_code)):

        latent_code_val, output_val, predicted_batch_effect = sess.run(
            [latent_code[i], output[i], removed_batch_effect], feed_dict={
                test_data_holder: test_data, test_exp_batch_idx_holder: batch_effect_idx})
        if i == 0:
            latent_fea = latent_code_val
            output_total = output_val
        else:
            latent_fea = np.concatenate([latent_fea, latent_code_val], 1)
            output_total = output_total + output_val

    output_val = output_total / len(latent_code)
    return latent_fea, output_val, predicted_batch_effect

'''Сама архитектура'''
#Поскольку количество строк одной матрицы связано с количеством столбцов другой, чтобы их можно было
#перемножать, нужно каждый раз - для каждого слоя - прописывать размерность выходной матрицы
def Inference(input_d, latent_code_dim, T, encoder_layers, decoder_layer, exp_batch_idx=[], re_use=False):
    '''
    The deep neural network structure of scScope
    Parameters:
			input_d:            gene expression matrix of dim n * m; n = number of cells, m = number of genes.
            latent_code_dim:    the dimension of features outputted by scScope.
            T:                  number of recurrent structures used in deep learning framework.
            encoder_layers:
            decoder_layer:
            exp_batch_idx:      if provided, experimental batch labels are stored in an n * batch_num matrix in one-hot format.
            re_use:             if re-use variables in training.
    Output:
            output_list:        outputs of decoder (y_c in the paper) in T recurrent structures.
            latent_code_list:   latent representations (h_c in the paper) in T recurrent structures.
            batch_effect_removal_layer:  experimental batch effects inferred by scScope.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''
#это мы определяем размерность
    input_shape = input_d.get_shape().as_list()
#это вроде индексацияя количества столбцов. Судя по всему, в столбцах лежат RNAseq-и. И мы дальше с помощью этого dim-a сможет
#изменять наши секи (добавлять/убирать)
    input_dim = input_shape[1]
#variable_scope - это создание группы. Нужно для того, чтобы было удобнее визуализировать в Tensor Board. Внутри лежат другие группы. 
    with tf.variable_scope('scScope') as scope_all:

        if re_use == True:
            scope_all.reuse_variables()

        latent_code_list = []
        output_list = []
        exp_batch_id_shape = exp_batch_idx.get_shape().as_list()
        exp_batch_dim = exp_batch_id_shape[1]
        with tf.variable_scope('batch_effect_removal'):
            batch_effect_para_weight = _variable_with_weight_decay('batch_effect_weight',
                                                                   [exp_batch_dim,
                                                                    input_dim],
                                                                   stddev=0, wd=0)

"""Мы работаем с огромным количеством данных. Внутри них есть batch-и - это группы данных, полученных в одной лабе при одних и тех же условиях.
Между batch-ами меняются конфаундеры, и они получаются сдвинутыми друг относительно друга. Если мы знаем, что данные должны быть гомогенными,
а они у нас гетерогенные, надо их как-то сдвинуть, то есть привести к одному мат ожиданию и одному стандартному отклонению. Обычно это делается
до того, как кормить сетку, то есть при подготовке данных. В случае Batch-нормализации мат ожидание равно 0, а дисперсия - 1. Это решает 
проблему "сдвига" данных при прохождении по внутренним слоям. Нам кажется странным, как работает у них этот этап.Они перемножает матрицы 
ожидаемого match_id и para_weight. para_weight - это тензор, содержащий веса. """

            batch_effect_removal_layer = tf.matmul(
                exp_batch_idx, batch_effect_para_weight)

        with tf.variable_scope('inference'):
            for i in range(T):
                #Первая итерация обучения:
                if i == 0:
                    encoder_layer_list_W = []
                    encoder_layer_list_b = []
                    if len(encoder_layers) > 0:
                        for l in range(len(encoder_layers)):
                            if l == 0:
                                encoder_layer_list_W.append(_variable_with_weight_decay('encoder_layer' + str(l),
                                                                                        [input_dim,
                                                                                         encoder_layers[l]],
                                                                                        stddev=0.1, wd=0))
                                encoder_layer_list_b.append(
                                    _variable_on_cpu('encoder_layer_bias' + str(l), [encoder_layers[l]],
                                                     tf.constant_initializer(0)))
                            else:
                                encoder_layer_list_W.append(_variable_with_weight_decay('encoder_layer' + str(l),
                                                                                        [encoder_layers[l - 1],
                                                                                         encoder_layers[l]],
                                                                                        stddev=0.1, wd=0))
                                encoder_layer_list_b.append(
                                    _variable_on_cpu('encoder_layer_bias' + str(l), [encoder_layers[l]],
                                                     tf.constant_initializer(0)))
                        latent_code_layer_input_dim = encoder_layers[-1]

                    else:
                        latent_code_layer_input_dim = input_dim

                    W_fea = _variable_with_weight_decay('latent_layer_weights',
                                                        [latent_code_layer_input_dim,
                                                         latent_code_dim],
                                                        stddev=0.1, wd=0)
                    b_fea = _variable_on_cpu('latent_layer_bias', [latent_code_dim],
                                             tf.constant_initializer(0))

                    decoder_layer_list_W = []
                    decoder_layer_list_b = []
                    if len(decoder_layer) > 0:
                        for l in range(len(decoder_layer)):
                            if l == 0:
                                decoder_layer_list_W.append(_variable_with_weight_decay('dencoder_layer' + str(l),
                                                                                        [latent_code_dim,
                                                                                         decoder_layer[l]],
                                                                                        stddev=0.1, wd=0))
                                decoder_layer_list_b.append(
                                    _variable_on_cpu('decoder_layer_bias' + str(l), [decoder_layer[l]],
                                                     tf.constant_initializer(0)))
                            else:
                                decoder_layer_list_W.append(_variable_with_weight_decay('dencoder_layer' + str(l),
                                                                                        [decoder_layer[l - 1],
                                                                                         decoder_layer[l]],
                                                                                        stddev=0.1, wd=0))
                                decoder_layer_list_b.append(
                                    _variable_on_cpu('decoder_layer_bias' + str(l), [decoder_layer[l]],
                                                     tf.constant_initializer(0)))
                        decoder_last_layer_dim = decoder_layer[-1]

                    else:
                        decoder_last_layer_dim = latent_code_dim

                    W_recon = _variable_with_weight_decay('reconstruction_layer_weights',
                                                          [decoder_last_layer_dim,
                                                           input_dim],
                                                          stddev=0.1, wd=0)
                    b_recon = _variable_on_cpu('reconstruction_layer_bias', [input_dim],
                                               tf.constant_initializer(0))
                    input_vec = tf.nn.relu(
                        input_d - batch_effect_removal_layer)
                #Все следующие после первой итерации обучения:
                else:
                    #Вторая итерация 
                    if i == 1:
                        W_feedback_1 = _variable_with_weight_decay('impute_layer_weights',
                                                                   [input_dim, 64],
                                                                   stddev=0.1, wd=0)
                        b_feedback_1 = _variable_on_cpu(
                            'impute_layer_bias', [64], tf.constant_initializer(0))

                        W_feedback_2 = _variable_with_weight_decay('impute_layer_weights2',
                                                                   [64, input_dim],
                                                                   stddev=0.1, wd=0)
                        b_feedback_2 = _variable_on_cpu(
                            'impute_layer_bias2', [input_dim], tf.constant_initializer(0))
                    #Здесь происходит магия импутации. Берем матрицу весов слоя импутации,
                    #умножаем на матрицу аутпута, прибавляем баес, считаем функцию активации.
                    intermediate_layer = tf.nn.relu(
                        tf.matmul(output, W_feedback_1) + b_feedback_1)
                    #Формируем слой импутации. (1 - tf.sign(input_d) = 0, если в позиции gene 
                    #expression matrix (input_d) есть значения и = 1, если там nan. 
                    imputation_layer = tf.multiply(
                        1 - tf.sign(input_d), (tf.matmul(intermediate_layer, W_feedback_2) + b_feedback_2))

                    input_vec = tf.nn.relu(
                        imputation_layer + input_d - batch_effect_removal_layer)

                intermedate_encoder_layer_list = []
                if len(encoder_layer_list_W) > 0:
                    for i in range(len(encoder_layer_list_W)):
                        if i == 0:
                            intermedate_encoder_layer_list.append(tf.nn.relu(
                                tf.matmul(input_vec, encoder_layer_list_W[i]) + encoder_layer_list_b[i]))
                        else:
                            intermedate_encoder_layer_list.append(tf.nn.relu(tf.matmul(
                                intermedate_encoder_layer_list[-1], encoder_layer_list_W[i]) + encoder_layer_list_b[i]))

                    intermedate_encoder_layer = intermedate_encoder_layer_list[-1]
                else:
                    intermedate_encoder_layer = input_vec

                latent_code = tf.nn.relu(
                    tf.matmul(intermedate_encoder_layer, W_fea) + b_fea)

                inter_decoder_layer_list = []

                if len(decoder_layer_list_W) > 0:
                    for i in range(len(decoder_layer_list_W)):
                        if i == 0:
                            inter_decoder_layer_list.append(tf.nn.relu(
                                tf.matmul(latent_code, decoder_layer_list_W[i]) + decoder_layer_list_b[i]))
                        else:
                            inter_decoder_layer_list.append(tf.nn.relu(tf.matmul(
                                inter_decoder_layer_list[-1], decoder_layer_list_W[i]) + decoder_layer_list_b[i]))
                    inter_decoder_layer = inter_decoder_layer_list[-1]
                else:
                    inter_decoder_layer = latent_code

                output = tf.nn.relu(
                    tf.matmul(inter_decoder_layer, W_recon) + b_recon)
                latent_code_list.append(latent_code)
                output_list.append(output)

        return output_list, latent_code_list, batch_effect_removal_layer


def tower_loss(scope, batch_data, use_mask, latent_code_dim, T, encoder_layers, decoder_layers, exp_batch_id,
               re_use_flag):
    '''
    Overall losses of scScope on multiple GPUs.
    Parameter:
            scope:              tensorflow name scope
            batch_data:         cell batch for calculating the loss
            use_mask:           flag indicating only use non-zero genes to calculate losses.
            latent_code_dim:    the dimension of features outputted by scScope.
            T:                  number of recurrent structures used in deep learning framework.
            encoder_layers:
            decoder_layers:
            exp_batch_id:
            re_use_flag:        if re-use variables in training.
    Output:
            total_loss:         total loss of multiple GPUs.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''

    layer_out, latent_code, batch_effect_removal_layer = Inference(
        batch_data, latent_code_dim, T, encoder_layers, decoder_layers, exp_batch_id, re_use=re_use_flag)

    _ = Cal_Loss(layer_out, batch_data, use_mask, batch_effect_removal_layer)

    losses = tf.get_collection('losses', scope)

    total_loss = tf.add_n(losses, name='total_loss')

    return total_loss


def Cal_Loss(outpout_layer_list, input_data, use_mask, removed_exp_batch_effect):
    '''
    Loss function of scScope.
    Parameter:
            outpout_layer_list:     encoder output of T recurrent structures in scScope.
            input_data:             original gene expression matrix inputted into scScope.
            use_mask:               flag indicating only use non-zero genes to calculate losses.
            removed_exp_batch_effect:
    Output:
            acc_loss:               loss function value.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''

    input_data_corrected = input_data - removed_exp_batch_effect

    if use_mask:
        val_mask = tf.sign(input_data_corrected)
    else:
        val_mask = tf.sign(input_data_corrected + 1)

    for i in range(len(outpout_layer_list)):
        layer_out = outpout_layer_list[i]
        if i == 0:
            reconstruct_loss = tf.reduce_mean(
                tf.norm(tf.multiply(val_mask, (layer_out - input_data_corrected))))
        else:
            reconstruct_loss = reconstruct_loss + \
                tf.reduce_mean(
                    tf.norm(tf.multiply(val_mask, (layer_out - input_data_corrected))))
    acc_loss = reconstruct_loss
    tf.add_to_collection('losses', acc_loss)
    return acc_loss


def scalable_cluster(latent_code,
                     kmeans_num=500,
                     cluster_num=400,
                     display_step=50,
                     phenograh_neighbor=30
                     ):
    '''
    Scalable  cluster:
    To perform graph clustering on large-scale data, we designed a scalable clustering strategy by combining k-means and PhenoGraph.
    Briefly, we divide cells into M (kmeans_num) groups of equal size and perform K-means (cluster_num) clustering on each group independently. 
	The whole dataset is split to M×K clusters and we only input the cluster centroids into PhenoGraph for graph clustering. 
	Finally, each cell is assigned to graph clusters according to the cluster labels of its nearest centroids.
    Parameters:
        latent_code:    n*m matrix; n = number of cells, m = dimension of feature representation.
        kmeans_num:     number of independent K-means clusterings used. This is also the subset number.
        cluster_num:    cluster number for each K-means clustering. This is also the "n_clusters" in KMeans function in sklearn package.
        display_step:   displaying the process of K-means clustering.
        phenograh_neighbor: "k" parameter in PhenoGraph package.
		
    Output:
            label:          Cluster labels for input cells.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''

    print('Scalable clustering:')
    print('Use %d subsets of cells for initially clustering...' % kmeans_num)

    stamp = np.floor(np.linspace(0, latent_code.shape[0], kmeans_num + 1))
    stamp = stamp.astype(int)

    cluster_ceter = np.zeros([kmeans_num * cluster_num, latent_code.shape[1]])
    mapping_sample_kmeans = np.zeros(latent_code.shape[0])

    for i in range(kmeans_num):

        low_bound = stamp[i]
        upp_bound = stamp[i + 1]
        sample_range = np.arange(low_bound, upp_bound)
        select_sample = latent_code[sample_range, :]

        kmeans = KMeans(n_clusters=cluster_num,
                        random_state=0).fit(select_sample)
        label = kmeans.labels_

        for j in range(cluster_num):
            cluster_sample_idx = np.nonzero(label == j)[0]
            cluster_sample = select_sample[cluster_sample_idx, :]
            cluster_ceter[i * cluster_num + j,
                          :] = np.mean(cluster_sample, axis=0)
            mapping_sample_kmeans[sample_range[cluster_sample_idx]
                                  ] = i * cluster_num + j

        if i % display_step == 0:
            print('\tK-means clustering for %d subset.' % i)

    print('Finish intially clustering by K-means.')
    print('Start PhenoGraph clustering...\n')

    label_pheno, graph, Q = phenograph.cluster(
        cluster_ceter, k=phenograh_neighbor, n_jobs=1)

    label = np.zeros(latent_code.shape[0])
    for i in range(label_pheno.max() + 1):
        center_index = np.nonzero(label_pheno == i)[0]
        for j in center_index:
            sample_index = np.nonzero(mapping_sample_kmeans == j)[
                0]  # samples belong to this center
            label[sample_index] = i
    print('Finish density down-sampling clustering.')

    return label

   "" exp_batch_idx_input:  (optional) n * batch_num matrix in one-hot format, if provided, experimental batch ids are used for batch correction."""


# Original

In [0]:
!pip3 install git+https://github.com/jacoblevine/phenograph.git
!pip3 install scanpy

  Cloning https://github.com/jacoblevine/phenograph.git to /tmp/pip-req-build-iyvlj7wd
  Running command git clone -q https://github.com/jacoblevine/phenograph.git /tmp/pip-req-build-iyvlj7wd
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8bq6p5i/wheels/78/4e/51/59f41302009330e31c4a2d0dc1cbbdad17f4379471d40ab9af
Successfully built PhenoGraph
     |████████████████████████████████| 266kB 2.9MB/s 
     |████████████████████████████████| 61kB 14.2MB/s 
     |████████████████████████████████| 3.3MB 21.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/21/17/02/39181727a4496366cc662141a0a29d497bbdb0d9149981ddcb
  Stored in directory: /root/.cache/pip/wheels/7f/e5/74/a94b1e21a75fbdff29343aeb9970279fbc9ab8a2a8cfaa87dc
Successfully built scanpy anndata
  Found existing installation: numba 0.40.1
    Uninstalling numba-0.40.1:
      Successfully uninstalled numba-0.40.1


In [0]:
# imports

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn.cluster import KMeans
import tensorflow as tf
import numpy as np
import phenograph

import random
import time

In [0]:
# ops.py

def _variable_with_weight_decay(name, shape, stddev, wd):
    """
    Helper to create an initialized Variable with weight decay.
    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.
    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.
    Returns:
      Variable Tensor
    Altschuler & Wu Lab 2018. 
    Software provided as is under Apache License 2.0.
    """

    dtype = tf.float32

    var = _variable_on_cpu(

        name,

        shape,

        tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))

    if wd != 0:

        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')

        tf.add_to_collection('losses', weight_decay)

    return var


def _variable_on_cpu(name, shape, initializer):
    """Helper to create a Variable stored on CPU memory.
    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable
    Returns:
      Variable Tensor
    Altschuler & Wu Lab 2018. 
    Software provided as is under Apache License 2.0.
    """

    with tf.device('/cpu:0'):

        dtype = tf.float32

        var = tf.get_variable(
            name, shape, initializer=initializer, dtype=dtype)

    return var


def average_gradients(tower_grads):
    """ Summarize the gradient calculated by each GPU.
    Altschuler & Wu Lab 2018. 
    Software provided as is under Apache License 2.0.
    """

    average_grads = []

    for grad_and_vars in zip(*tower_grads):

        # Note that each grad_and_vars looks like the following:

        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))

        grads = []

        for g, var1 in grad_and_vars:

            # Add 0 dimension to the gradients to represent the tower.

            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.

            grads.append(expanded_g)

        # Average over the 'tower' dimension.

        grad = tf.concat(axis=0, values=grads)

        grad = tf.reduce_mean(grad, 0)

        v = grad_and_vars[0][1]

        grad_and_var = (grad, v)

        average_grads.append(grad_and_var)

    return average_grads

In [0]:
# __init__.py

# scScope is a deep-learning based approach designed to identify cell-type composition from large-scale scRNA-seq profiles.


def train(train_data_set,
          latent_code_dim,
          use_mask=True,
          batch_size=64,
          max_epoch=100,
          epoch_per_check=100,
          T=2,
          exp_batch_idx_input=[],
          encoder_layers=[],
          decoder_layers=[],
          learning_rate=0.0001,
          beta1=0.05,
          num_gpus=1
          ):
    '''
    scScope training:
	  This function is used to train the scScope model on gene expression data
    Parameters:
      train_data_set:       gene expression matrix of dim n * m; n = number of cells, m = number of genes.
      latent_code_dim:      feature dimension outputted by scScope.
      batch_size:           number of cells used in each training iteration.
      max_epoch:            maximal epoch used in training.
      epoch_per_check:      step to display current loss.
      T:                    depth of recurrence used in deep learning framework.
      use_mask:             flag indicating whether to use only non-zero entries in calculating losses.
      learning_rate:        step length in gradient descending algorithm.
      beta1:                beta1 parameter in AdamOptimizer.
      num_gpus:             number of gpus used for training in parallel.
      exp_batch_idx_input:  (optional) n * batch_num matrix in one-hot format, if provided, experimental batch ids are used for batch correction.
      encoder_layers:       network structure for encoder layers of the autoencoder; e.g. [64,128] means adding two layers with 64 and 128 nodes between the input and hidden features
      decoder_layers:       network structure for decoder layers of the autoencoder; e.g. [64,128] means adding two layers with 64 and 128 nodes between the hidden feature and the output layer
    Output:
      model: a dataframe of scScope outputs with keys:
            'latent_code_session':      tensorflow session used in training.
            'test_input':               tensorflow dataholder for test data.
            'test_exp_batch_idx':       tensorflow dataholder for experimental batch label.
            'imputated_output':         imputed gene expressions.
            'latent_code':              latent features by scScope.
            'removed_batch_effect':     correcting layer learning by scScope.
    Altschuler & Wu Lab 2018. 
    Software provided as is under Apache License 2.0.
    '''

    batch_size = int(batch_size*num_gpus)
    learning_rate = learning_rate*num_gpus

    if len(exp_batch_idx_input) == 0:
        exp_batch_idx_input = np.zeros((np.shape(train_data_set)[0], 1))
        consider_exp_batch = False
    else:
        consider_exp_batch = True

    with tf.Graph().as_default(), tf.device('/cpu:0'):

        train_data = tf.placeholder(
            tf.float32, [batch_size, np.shape(train_data_set)[1]])
        exp_batch_idx = tf.placeholder(tf.float32,
                                       [batch_size, np.shape(exp_batch_idx_input)[1]])

        # Create an optimizer that performs gradient descent.

        opt = tf.train.AdamOptimizer(learning_rate, beta1=beta1)

        # Calculate the gradients on models deployed on each GPU then summarize the gradients.
        tower_grads = []
        tower_grads2 = []

        with tf.variable_scope(tf.get_variable_scope()):

            for i in range(num_gpus):

                print('Building Computational Graph on GPU-'+str(i))

                with tf.device('/gpu:%d' % (i+1)):

                    with tf.name_scope('%s_%d' % ('tower', i)) as scope:

                        itv = int(batch_size/num_gpus)

                        if i == 0:

                            re_use_flag = False

                        else:

                            re_use_flag = True

                        loss = tower_loss(scope,
                                          train_data[(i) *
                                                     itv:(i + 1) * itv, :],
                                          use_mask,
                                          latent_code_dim,
                                          T,
                                          encoder_layers,
                                          decoder_layers,
                                          exp_batch_idx[(
                                              i) * itv:(i + 1) * itv, :],
                                          re_use_flag)

                        tf.get_variable_scope().reuse_variables()

                        t_vars = tf.trainable_variables()

                        inference_para = [
                            var for var in t_vars if 'inference' in var.name]
                        grads = opt.compute_gradients(loss, inference_para)
                        tower_grads.append(grads)

                        if consider_exp_batch:
                            exp_batch_effect_para = [
                                var for var in t_vars if 'batch_effect_removal' in var.name]
                            grads2 = opt.compute_gradients(
                                loss, exp_batch_effect_para)
                            tower_grads2.append(grads2)

                        # Save gradients from different GPUs.

        # Summarize gradients from multiple GPUs.
        grads = average_gradients(tower_grads)
        apply_gradient_op = opt.apply_gradients(grads)
        train_op = apply_gradient_op

        if consider_exp_batch:
            grads2 = average_gradients(tower_grads2)
            apply_gradient_op2 = opt.apply_gradients(grads2)
            train_op2 = apply_gradient_op2

        init = tf.global_variables_initializer()

        # Configuration of GPUs.
        config_ = tf.ConfigProto()

        config_.gpu_options.allow_growth = True

        config_.allow_soft_placement = True

        sess = tf.Session(config=config_)

        sess.run(init)

        total_data_size = np.shape(train_data_set)[0]

        total_sample_list = list(range(total_data_size))

        reconstruction_error = []

        start = time.time()
        for step in range(1, max_epoch+1):

            total_cnt = total_data_size/(batch_size)

            for itr_cnt in range(int(total_cnt)):

                sel_pos = random.sample(total_sample_list, batch_size)

                cur_data = train_data_set[sel_pos, :]
                cur_exp_batch_idx = exp_batch_idx_input[sel_pos, :]

                sess.run(train_op,
                         feed_dict={train_data: cur_data,
                                    exp_batch_idx: cur_exp_batch_idx})
                if consider_exp_batch:
                    sess.run(train_op2,
                             feed_dict={train_data: cur_data,
                                        exp_batch_idx: cur_exp_batch_idx})

            if step % epoch_per_check == 0 and step > 0:

                all_input = tf.placeholder(
                    tf.float32, [np.shape(train_data_set)[0], np.shape(train_data_set)[1]])
                exp_batch_idx_all = tf.placeholder(
                    tf.float32, [np.shape(exp_batch_idx_input)[0], np.shape(exp_batch_idx_input)[1]])

                layer_output, train_latent_code, _ = Inference(
                    all_input, latent_code_dim, T, encoder_layers, decoder_layers, exp_batch_idx_all, re_use=True)

                train_code_val, layer_output_val = sess.run(
                    [train_latent_code[-1], layer_output[-1]], feed_dict={all_input: train_data_set, exp_batch_idx_all: exp_batch_idx_input})

                mask = np.sign(train_data_set)
                recon_error = np.linalg.norm(np.multiply(mask, layer_output_val)-np.multiply(
                    mask, train_data_set))/np.linalg.norm(np.multiply(mask, train_data_set))
                reconstruction_error.append(recon_error)
                print("Finisheded epoch：" + str(step))
                print('Current reconstruction error is: '+str(recon_error))

                if len(reconstruction_error) >= 2:
                    if (abs(reconstruction_error[-1] - reconstruction_error[-2])/reconstruction_error[-2] < 1e-3):
                        break

        Model = {}

        test_data_holder = tf.placeholder(
            tf.float32, [None, np.shape(train_data_set)[1]])
        test_exp_batch_idx = tf.placeholder(
            tf.float32, [None, np.shape(exp_batch_idx_input)[1]])

        test_layer_out, test_latent_code, removed_batch_effect = Inference(
            test_data_holder, latent_code_dim, T, encoder_layers, decoder_layers, test_exp_batch_idx, re_use=True)

        Model['latent_code_session'] = sess
        Model['test_input'] = test_data_holder
        Model['test_exp_batch_idx'] = test_exp_batch_idx
        Model['Imputated_output'] = test_layer_out
        Model['latent_code'] = test_latent_code
        Model['removed_batch_effect'] = removed_batch_effect

        duration = time.time()-start
        print('Finish training ' + str(len(train_data_set)) + ' samples after '+str(step)+' epochs. The total training time is ' +
              str(duration)+' seconds.')

        return Model


def predict(test_data, model, batch_effect=[]):
    '''
    Make predications using the learned scScope model.
    Parameter:
        test_data:      input gene expression matrix.
        model:          pre-trained scScope model.
    Output:
        latent_fea:             scScope features output.
        output_val:             gene expressions with imputations.
        predicted_batch_effect: batch effects inferenced by scScope, if experimental batches exist.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''

    sess = model['latent_code_session']
    test_data_holder = model['test_input']
    test_exp_batch_idx_holder = model['test_exp_batch_idx']
    output = model['Imputated_output']
    latent_code = model['latent_code']
    removed_batch_effect = model['removed_batch_effect']
    if len(batch_effect) == 0:
        batch_effect_idx = np.zeros((np.shape(test_data)[0], 1))
    else:
        batch_effect_idx = batch_effect

    for i in range(len(latent_code)):

        latent_code_val, output_val, predicted_batch_effect = sess.run([latent_code[i], output[i], removed_batch_effect], feed_dict={
            test_data_holder: test_data, test_exp_batch_idx_holder: batch_effect_idx})
        if i == 0:
            latent_fea = latent_code_val
        else:
            latent_fea = np.concatenate([latent_fea, latent_code_val], 1)

    return latent_fea, output_val, predicted_batch_effect


def Inference(input_d, latent_code_dim, T, encoder_layers, decoder_layer, exp_batch_idx=[], re_use=False):
    '''
    The deep neural network structure of scScope.
    Parameters:
        input_d:            gene expression matrix of dim n * m; n = number of cells, m = number of genes.
        latent_code_dim:    the dimension of features outputted by scScope.
        T:                  number of recurrent structures used in deep learning framework.
        encoder_layers:     the network structure for encoder layers of the autoencoder.
        decoder_layers:     the network structure for decoder layers of the autoencoder.
        exp_batch_idx:      if provided, experimental batch labels are stored in an n * batch_num matrix in one-hot format.
        re_use:             if re-use variables in training.
    Output:
        output_list:        outputs of decoder (y_c in the paper) in T recurrent structures.
        latent_code_list:   latent representations (h_c in the paper) in T recurrent structures.
        batch_effect_removal_layer:  experimental batch effects inferred by scScope.
    Altschuler & Wu Lab 2018.
    Software provided as is under Apache License 2.0.
    '''

    input_shape = input_d.get_shape().as_list()

    input_dim = input_shape[1]

    with tf.variable_scope('scScope') as scope_all:

        if re_use == True:

            scope_all.reuse_variables()

        latent_code_list = []
        output_list = []
        exp_batch_id_shape = exp_batch_idx.get_shape().as_list()
        exp_batch_dim = exp_batch_id_shape[1]
        with tf.variable_scope('batch_effect_removal'):
            batch_effect_para_weight = _variable_with_weight_decay('batch_effect_weight', [exp_batch_dim, input_dim],
                                                                   stddev=0, wd=0)

            batch_effect_removal_layer = tf.matmul(
                exp_batch_idx, batch_effect_para_weight)

        with tf.variable_scope('inference'):
            for i in range(T):
                if i == 0:
                    encoder_layer_list_W = []
                    encoder_layer_list_b = []
                    if len(encoder_layers) > 0:
                        for l in range(len(encoder_layers)):
                            if l == 0:
                                encoder_layer_list_W.append(_variable_with_weight_decay('encoder_layer' + str(l),
                                                                                        [input_dim,
                                                                                         encoder_layers[l]],
                                                                                        stddev=0.1, wd=0))
                                encoder_layer_list_b.append(_variable_on_cpu('encoder_layer_bias' + str(l), [encoder_layers[l]],
                                                                             tf.constant_initializer(0)))
                            else:
                                encoder_layer_list_W.append(_variable_with_weight_decay('encoder_layer' + str(l),
                                                                                        [encoder_layers[l-1],
                                                                                         encoder_layers[l]],
                                                                                        stddev=0.1, wd=0))
                                encoder_layer_list_b.append(_variable_on_cpu('encoder_layer_bias' + str(l), [encoder_layers[l]],
                                                                             tf.constant_initializer(0)))
                        latent_code_layer_input_dim = encoder_layers[-1]

                    else:
                        latent_code_layer_input_dim = input_dim

                    W_fea = _variable_with_weight_decay('latent_layer_weights',
                                                        [latent_code_layer_input_dim,
                                                         latent_code_dim],
                                                        stddev=0.1, wd=0)
                    b_fea = _variable_on_cpu('latent_layer_bias', [latent_code_dim],
                                             tf.constant_initializer(0))

                    decoder_layer_list_W = []
                    decoder_layer_list_b = []
                    if len(decoder_layer) > 0:
                        for l in range(len(decoder_layer)):
                            if l == 0:
                                decoder_layer_list_W.append(_variable_with_weight_decay('dencoder_layer' + str(l),
                                                                                        [latent_code_dim,
                                                                                         decoder_layer[l]],
                                                                                        stddev=0.1, wd=0))
                                decoder_layer_list_b.append(
                                    _variable_on_cpu('decoder_layer_bias' + str(l), [decoder_layer[l]],
                                                     tf.constant_initializer(0)))
                            else:
                                decoder_layer_list_W.append(_variable_with_weight_decay('dencoder_layer' + str(l),
                                                                                        [decoder_layer[l - 1],
                                                                                         decoder_layer[l]],
                                                                                        stddev=0.1, wd=0))
                                decoder_layer_list_b.append(
                                    _variable_on_cpu('decoder_layer_bias' + str(l), [decoder_layer[l]],
                                                     tf.constant_initializer(0)))
                        decoder_last_layer_dim = decoder_layer[-1]

                    else:
                        decoder_last_layer_dim = latent_code_dim

                    W_recon = _variable_with_weight_decay('reconstruction_layer_weights',
                                                          [decoder_last_layer_dim,
                                                           input_dim],
                                                          stddev=0.1, wd=0)
                    b_recon = _variable_on_cpu('reconstruction_layer_bias', [input_dim],
                                               tf.constant_initializer(0))
                    input_vec = tf.nn.relu(input_d-batch_effect_removal_layer)
                else:

                    if i == 1:
                        W_feedback_1 = _variable_with_weight_decay('impute_layer_weights',
                                                                   [input_dim, 64],
                                                                   stddev=0.1, wd=0)
                        b_feedback_1 = _variable_on_cpu(
                            'impute_layer_bias', [64], tf.constant_initializer(0))

                        W_feedback_2 = _variable_with_weight_decay('impute_layer_weights2',
                                                                   [64, input_dim],
                                                                   stddev=0.1, wd=0)
                        b_feedback_2 = _variable_on_cpu(
                            'impute_layer_bias2', [input_dim], tf.constant_initializer(0))
                        # else:
                        #     W_feedback_2 = ops._variable_with_weight_decay('impute_layer_weights2',
                        #                                                    [input_dim, input_dim],
                        #                                                    stddev=0.1, wd=0)
                        #     b_feedback_2 = ops._variable_on_cpu(
                        #         'impute_layer_bias2', [input_dim], tf.constant_initializer(0))

                    # if input_dim>500:
                    intermediate_layer = tf.nn.relu(
                        tf.matmul(output, W_feedback_1) + b_feedback_1)
                    imputation_layer = tf.multiply(
                        1-tf.sign(input_d),  (tf.matmul(intermediate_layer, W_feedback_2)+b_feedback_2))
                    # else:
                    #     imputation_layer = tf.multiply(1 - tf.sign(input_d),
                    #                                    (tf.matmul(output, W_feedback_2) + b_feedback_2))

                    input_vec = tf.nn.relu(
                        imputation_layer+input_d-batch_effect_removal_layer)

                intermedate_encoder_layer_list = []
                if len(encoder_layer_list_W) > 0:
                    for i in range(len(encoder_layer_list_W)):
                        if i == 0:
                            intermedate_encoder_layer_list.append(tf.nn.relu(
                                tf.matmul(input_vec, encoder_layer_list_W[i])+encoder_layer_list_b[i]))
                        else:
                            intermedate_encoder_layer_list.append(tf.nn.relu(tf.matmul(
                                intermedate_encoder_layer_list[-1], encoder_layer_list_W[i])+encoder_layer_list_b[i]))

                    intermedate_encoder_layer = intermedate_encoder_layer_list[-1]
                else:
                    intermedate_encoder_layer = input_vec

                latent_code = tf.nn.relu(
                    tf.matmul(intermedate_encoder_layer, W_fea)+b_fea)

                inter_decoder_layer_list = []

                if len(decoder_layer_list_W) > 0:
                    for i in range(len(decoder_layer_list_W)):
                        if i == 0:
                            inter_decoder_layer_list.append(tf.nn.relu(
                                tf.matmul(latent_code, decoder_layer_list_W[i])+decoder_layer_list_b[i]))
                        else:
                            inter_decoder_layer_list.append(tf.nn.relu(tf.matmul(
                                inter_decoder_layer_list[-1], decoder_layer_list_W[i])+decoder_layer_list_b[i]))
                    inter_decoder_layer = inter_decoder_layer_list[-1]
                else:
                    inter_decoder_layer = latent_code

                output = tf.nn.relu(
                    tf.matmul(inter_decoder_layer, W_recon)+b_recon)
                latent_code_list.append(latent_code)
                output_list.append(output)

        return output_list, latent_code_list, batch_effect_removal_layer


def tower_loss(scope, batch_data, use_mask, latent_code_dim, T, encoder_layers, decoder_layers, exp_batch_id,
               re_use_flag):
    '''
        Overall losses of scScope on multiple GPUs.
        Parameter:
                scope:              tensorflow name scope
                batch_data:         cell batch for calculating the loss
                use_mask:           flag indicating only use non-zero genes to calculate losses.
                latent_code_dim:    the dimension of features outputted by scScope.
                T:                  number of recurrent structures used in deep learning framework.
                encoder_layers:     the network structure for encoder layers of the autoencoder.
                decoder_layers:     the network structure for decoder layers of the autoencoder.
                exp_batch_idx:      if provided, experimental batch labels are stored in an n * batch_num matrix in one-hot format.
                re_use_flag:        if re-use variables in training.
        Output:
                total_loss:         total loss of multiple GPUs.
        Altschuler & Wu Lab 2018.
        Software provided as is under Apache License 2.0.
        '''

    layer_out, latent_code, batch_effect_removal_layer = Inference(
        batch_data, latent_code_dim, T, encoder_layers, decoder_layers, exp_batch_id, re_use=re_use_flag)

    _ = Cal_Loss(layer_out, batch_data, use_mask, batch_effect_removal_layer)

    losses = tf.get_collection('losses', scope)

    total_loss = tf.add_n(losses, name='total_loss')

    return total_loss


def Cal_Loss(outpout_layer_list, input_data, use_mask, removed_exp_batch_effect):
    '''
        Loss function of scScope.
        Parameter:
                outpout_layer_list:     encoder output of T recurrent structures in scScope.
                input_data:             original gene expression matrix inputted into scScope.
                use_mask:               flag indicating only use non-zero genes to calculate losses.
                removed_exp_batch_effect: removed exeperimental batch effects
        Output:
                acc_loss:               loss function value.
        Altschuler & Wu Lab 2018.
        Software provided as is under Apache License 2.0.
        '''

    input_data_corrected = input_data - removed_exp_batch_effect

    if use_mask:
        val_mask = tf.sign(input_data_corrected)
    else:
        val_mask = tf.sign(input_data_corrected + 1)

    for i in range(len(outpout_layer_list)):
        layer_out = outpout_layer_list[i]
        if i == 0:
            reconstruct_loss = tf.reduce_mean(
                tf.norm(tf.multiply(val_mask, (layer_out - input_data_corrected))))
        else:
            reconstruct_loss = reconstruct_loss + \
                tf.reduce_mean(
                    tf.norm(tf.multiply(val_mask, (layer_out - input_data_corrected))))
    acc_loss = reconstruct_loss
    tf.add_to_collection('losses', acc_loss)
    return acc_loss


def scalable_cluster(latent_code,
                     kmeans_num=500,
                     cluster_num=400,
                     display_step=50,
                     phenograh_neighbor=30
                     ):
    '''
    Scalable  cluster:
    To perform graph clustering on large-scale data, we designed a scalable clustering strategy by combining k-means and PhenoGraph.
    Briefly, we divide cells into M (kmeans_num) groups of equal size and perform K-means (cluster_num) clustering on each group independently. 
	The whole dataset is split to M×K clusters and we only input the cluster centroids into PhenoGraph for graph clustering. 
	Finally, each cell is assigned to graph clusters according to the cluster labels of its nearest centroids.
    Parameters:
        latent_code:    n*m matrix; n = number of cells, m = dimension of feature representation.
        kmeans_num:     number of independent K-means clusterings used. This is also the subset number.
        cluster_num:    cluster number for each K-means clustering. This is also the "n_clusters" in KMeans function in sklearn package.
        display_step:   displaying the process of K-means clustering.
        phenograh_neighbor: "k" parameter in PhenoGraph package.
    Output:
        Cluster labels for input cells.
    Altschuler & Wu Lab 2018. 
    Software provided as is under Apache License 2.0.
    '''

    print('Scalable clustering:')
    print('Use %d subsets of cells for initially clustering...' % kmeans_num)

    stamp = np.floor(np.linspace(0, latent_code.shape[0], kmeans_num + 1))
    stamp = stamp.astype(int)

    cluster_ceter = np.zeros([kmeans_num * cluster_num, latent_code.shape[1]])
    mapping_sample_kmeans = np.zeros(latent_code.shape[0])

    for i in range(kmeans_num):

        low_bound = stamp[i]
        upp_bound = stamp[i + 1]
        sample_range = np.arange(low_bound, upp_bound)
        select_sample = latent_code[sample_range, :]

        kmeans = KMeans(n_clusters=cluster_num,
                        random_state=0).fit(select_sample)
        label = kmeans.labels_

        for j in range(cluster_num):
            cluster_sample_idx = np.nonzero(label == j)[0]
            cluster_sample = select_sample[cluster_sample_idx, :]
            cluster_ceter[i * cluster_num + j,
                          :] = np.mean(cluster_sample, axis=0)
            mapping_sample_kmeans[sample_range[cluster_sample_idx]
                                  ] = i * cluster_num + j

        if i % display_step == 0:
            print('\tK-means clustering for %d subset.' % i)

    print('Finish intially clustering by K-means.')
    print('Start PhenoGraph clustering...\n')

    label_pheno, graph, Q = phenograph.cluster(
        cluster_ceter, k=phenograh_neighbor, n_jobs=1)

    label = np.zeros(latent_code.shape[0])
    for i in range(label_pheno.max() + 1):
        center_index = np.nonzero(label_pheno == i)[0]
        for j in center_index:
            sample_index = np.nonzero(mapping_sample_kmeans == j)[
                0]  # samples belong to this center
            label[sample_index] = i
    print('Finish density down-sampling clustering.')

    return label

## Starting

In [0]:
import rpy2
%load_ext rpy2.ipython

In [0]:
%%R
install.packages('splatter')
library(splatter)

simulate <- function(nGroups=1, nGenes=1000, batchCells=2000, dropout=3)
{
  if (nGroups > 1) method <- 'groups'
  else             method <- 'single'
  
  group.prob <- rep(1, nGroups) / nGroups
  
  # new splatter requires dropout.type
  if ('dropout.type' %in% slotNames(newSplatParams())) {
    if (dropout)
      dropout.type <- 'experiment'
    else
      dropout.type <- 'none'
    
    sim <- splatSimulate(group.prob=group.prob, nGenes=nGenes, batchCells=batchCells,
                         dropout.type=dropout.type, method=method,
                         seed=0, dropout.shape=-1, dropout.mid=dropout)
    
  } else {
    sim <- splatSimulate(group.prob=group.prob, nGenes=nGenes, batchCells=batchCells,
                         dropout.present=!dropout, method=method,
                         seed=0, dropout.shape=-1, dropout.mid=dropout)        
  }
  
  counts     <- as.data.frame(t(counts(sim)))
  truecounts <- as.data.frame(t(assays(sim)$TrueCounts))
  
  dropout    <- assays(sim)$Dropout
  mode(dropout) <- 'integer'
  
  cellinfo   <- as.data.frame(colData(sim))
  geneinfo   <- as.data.frame(rowData(sim))
  
  list(counts=counts,
       cellinfo=cellinfo,
       geneinfo=geneinfo,
       truecounts=truecounts)
}

cell_num<-2000
drop_out<-4
group<-5
sim <- simulate(nGroups=group,batchCells=cell_num,dropout = drop_out)

counts <- sim$counts
geneinfo <- sim$geneinfo
cellinfo <- sim$cellinfo
truecounts <- sim$truecounts

write.csv(counts, file =paste("./counts.csv",sep="") )
write.csv(cellinfo, file =paste( "./cellinfo.csv",sep=""))
write.csv(geneinfo, file = paste("./geneinf.csv",sep=""))

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)



Error in library(splatter) : there is no package called ‘splatter’


/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: In addition: 
  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: package ‘splatter’ is not available (for R version 3.6.0) 

  warnings.warn(x, RRuntimeWarning)


In [0]:
#demo.py

'''The goal of this demo is to show how to identify cell subpopulations based on latent
representations of gene expression learned by scScope.'''
import pandas as pd
import phenograph
import pickle
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# For this demo we normalize data using scanpy which is not a required package for scScope.
# To install, use: pip install scanpy
import scanpy.api as sc


def RUN_MAIN():

    # 1. Load gene expression matrix of simulated data
    # gene expression with simulated dropouts
    counts_drop = pd.read_csv('counts_1.csv', header=0, index_col=0)
    # ground trouth subpopulation assignment
    cellinfo = pd.read_csv('cellinfo_1.csv', header=0, index_col=0)

    group = cellinfo.Group
    label_ground_truth = []
    for g in group:
        g = int(g.split('Group')[1])
        label_ground_truth.append(g)

    # 2. Normalize gene expression based on scanpy (normalize each cell to have same library size)
    # matrix of cells x genes
    gene_expression = sc.AnnData(counts_drop.values)
    # normalize each cell to have same count number
    sc.pp.normalize_per_cell(gene_expression)
    # update datastructure to use normalized data
    gene_expression = gene_expression.X

    latent_dim = 50

    # 3. scScope learning
    if gene_expression.shape[0] >= 100000:
        DI_model = train(
            gene_expression, latent_dim, T=2, batch_size=512, max_epoch=10, num_gpus=1, epoch_per_check=25)
    else:
        DI_model = train(
            gene_expression, latent_dim, T=2, batch_size=64, max_epoch=300, num_gpus=1, epoch_per_check=25)

    # 4. latent representations and imputed expressions
    latent_code, imputed_val, _ = predict(
        gene_expression, DI_model)

    # 5. graph clustering
    if latent_code.shape[0] <= 10000:
        label, _, _ = phenograph.cluster(latent_code)
    else:
        label = scalable_cluster(latent_code)

    # evaluate
    ARI = adjusted_rand_score(label, label_ground_truth)
    print(ARI)

    X_embedded = TSNE(n_components=2).fit_transform(latent_code)

    # visualization of the subpopulation using tSNE
    plt.figure()
    for i in range(5):
        idx = np.nonzero(label == i)[0]
        plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], alpha=0.5)
    plt.show()


if __name__ == '__main__':
    RUN_MAIN()

ModuleNotFoundError: ignored

In [0]:
test_data = test

latent_code, imputed_val = prerdict(test_data, DI_model, batch_effect=[])
label, _,  _ = phenograph.cluster(latent_code)